In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import re
import warnings
#warnings.simplefilter('ignore')

#Specifying which teams and years to include in data set
Teams = ['CHC', 'CHW', 'LAA', 'LAD', 'NYM', 'NYY', 'OAK', 'SFG']
Years = []
for year in range(2010,2020):
    Years.append(str(year))

list_of_df = list()

#Creating dataframe
for team in Teams:
    for year in Years:
        #Will retrieve data from each year for each team
        url = 'https://www.baseball-reference.com/teams/' + team + '/' + year +'-schedule-scores.shtml'
        dfname = team + '_' + year
        html = requests.get(url).content
        df_list = pd.read_html(html)
        df = df_list[-1]
        
        #Formatting data table
        #rename columns
        df.rename(columns={"Gm#": "GM_Num", "Unnamed: 4": "Home", "Tm": "Team", "D/N": "Night"}, inplace = True)
        #turn home, game win, and night into dummy variables
        df['Home'] = df['Home'].apply(lambda x: 0 if x == '@' else 1)
        df['Game_Win'] = df['W/L'].astype(str).str[0]
        df['Game_Win'] = df['Game_Win'].apply(lambda x: 0 if x == 'L' else 1)
        df['Night'] = df['Night'].apply(lambda x: 1 if x == 'N' else 0)
        #quantify streak as number
        df['Streak'] = df['Streak'].apply(lambda x: -1*len(x) if '-' in x else len(x))
        df.drop('Unnamed: 2', axis=1, inplace = True)
        df.drop('Orig. Scheduled', axis=1, inplace = True)
        df.drop('Win', axis=1, inplace = True)
        df.drop('Loss', axis=1, inplace = True)
        df.drop('Save', axis=1, inplace = True)
        #Drop rows that do not have data
        df = df[df['GM_Num'].str.isdigit()]
        #Convert W-L column to 4 new numeric columns: Wins, Losses, Net Wins (Wins - Losses), Win Percentage (Wins/Total Games)
        WL = df["W-L"].str.split("-", n = 1, expand = True)
        df["Wins"] = WL[0].astype(dtype=np.int64)
        df["Losses"] = WL[1].astype(dtype=np.int64)
        df['Net_Wins'] = df['Wins'] - df['Losses']
        df['Win_Per'] = df['Wins']/(df['Wins']+df['Losses'])
        #Turn date into datetime object
        DayDate = df['Date'].str.split(", ", n = 1, expand = True)
        df['DayOfWeek'] = DayDate[0]
        df['Date'] = DayDate[1] + ', ' + year
        df['Date'] = [re.sub("\s\(\d+\)", "", str(x)) for x in df['Date']]
        df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y')
        #Add to list which will be turned into a dataframe
        list_of_df.append(df)

#Create dataframe
bbattend = pd.concat(list_of_df)
#bbattend       

/Users/laurel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/laurel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/laurel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

In [2]:
#Create Year Variable for Matching Later
bbattend['Year'] = bbattend.Date.dt.year

#Specify what the same-market team is for matching
bbattend['Same_Mkt_Team'] = bbattend.apply(lambda _: '', axis=1)
bbattend['Same_Mkt_Team'][bbattend['Team'].str.contains('LAA')] = 'LAD'
bbattend['Same_Mkt_Team'][bbattend['Team'].str.contains('LAD')] = 'LAA'
bbattend['Same_Mkt_Team'][bbattend['Team'].str.contains('NYY')] = 'NYM'
bbattend['Same_Mkt_Team'][bbattend['Team'].str.contains('NYM')] = 'NYY'
bbattend['Same_Mkt_Team'][bbattend['Team'].str.contains('CHW')] = 'CHC'
bbattend['Same_Mkt_Team'][bbattend['Team'].str.contains('CHC')] = 'CHW'
bbattend['Same_Mkt_Team'][bbattend['Team'].str.contains('OAK')] = 'SFG'
bbattend['Same_Mkt_Team'][bbattend['Team'].str.contains('SFG')] = 'OAK'

#Create day of week dummy variables
WeekDays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for i in WeekDays:
    bbattend[i] = bbattend.apply(lambda _: '', axis=1)
    bbattend[i] = bbattend['DayOfWeek'].apply(lambda x: 1 if x == i else 0)

#Create game_id which will be used to delete duplicates later
bbattend['game_id'] = bbattend['Team'] + bbattend['Date'].astype(str)
#bbattend

/Users/laurel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/laurel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/laurel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/laurel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A valu

In [3]:
# Create merged table
# Will match all dates of games of team with dates within same year of teams from same-market team 
merged = bbattend.merge(
    bbattend[["Date", "Year", "Team", "Net_Wins", "Win_Per","Streak", 'Wins', 'Losses']],
    how="inner",
    left_on=["Year", "Same_Mkt_Team"],
    right_on=["Year", "Team"],
    suffixes=('', '_Same_Mkt_Team')
)

#Measure how far apart the dates of the games are
merged["date_diff"] = (merged.Date - merged.Date_Same_Mkt_Team).dt.days
#Only keep the dates of same-market team that occurred before the date of home team's game
merged = merged[merged['date_diff'] > 0]

In [4]:
#Sort by date_diff so closest dates appear first
merged.sort_values(by='date_diff', inplace = True)

#Only keep first game_id which will include the data of the same-market team for the closest date before the game
merged.drop_duplicates(subset =['game_id'], keep = 'first', inplace = True)


merged.sort_values(by=['Team', 'Date'], inplace = True)
merged.head(20)

,GM_Num,Date,Team,Home,Opp,W/L,R,RA,Inn,W-L,...,Sunday,game_id,Date_Same_Mkt_Team,Team_Same_Mkt_Team,Net_Wins_Same_Mkt_Team,Win_Per_Same_Mkt_Team,Streak_Same_Mkt_Team,Wins_Same_Mkt_Team,Losses_Same_Mkt_Team,date_diff
162,2,2010-04-07,CHC,0,ATL,L,2,3,NaN,0-2,...,0,CHC2010-04-07,2010-04-05,CHW,1,1.000000,1,1,0,2
325,3,2010-04-08,CHC,0,ATL,W,2,0,NaN,1-2,...,0,CHC2010-04-08,2010-04-07,CHW,0,0.500000,-1,1,1,1
488,4,2010-04-09,CHC,0,CIN,L,4,5,NaN,1-3,...,0,CHC2010-04-09,2010-04-08,CHW,-1,0.333333,-2,1,2,1
651,5,2010-04-10,CHC,0,CIN,W,4,3,NaN,2-3,...,0,CHC2010-04-10,2010-04-09,CHW,-2,0.250000,-3,1,3,1
814,6,2010-04-11,CHC,0,CIN,L,1,3,NaN,2-4,...,1,CHC2010-04-11,2010-04-10,CHW,-3,0.200000,-4,1,4,1
977,7,2010-04-12,CHC,1,MIL,W,9,5,NaN,3-4,...,0,CHC2010-04-12,2010-04-11,CHW,-2,0.333333,1,2,4,1
1141,8,2010-04-14,CHC,1,MIL,W,7,6,NaN,4-4,...,0,CHC2010-04-14,2010-04-13,CHW,-2,0.375000,-1,3,5,1
1304,9,2010-04-15,CHC,1,MIL,L,6,8,NaN,4-5,...,0,CHC2010-04-15,2010-04-14,CHW,-1,0.444444,1,4,5,1
1467,10,2010-04-16,CHC,1,HOU,W,7,2,NaN,5-5,...,0,CHC2010-04-16,2010-04-15,CHW,-2,0.400000,-1,4,6,1
1630,11,2010-04-17,CHC,1,HOU,L,3,4,NaN,5-6,...,0,CHC2010-04-17,2010-04-16,CHW,-3,0.363636,-2,4,7,1


In [5]:
#merged.to_csv('bbattend.csv')

In [6]:
#create df with just home games
homegames = merged[merged.Home == 1]
#homegames.to_csv('bbattendhome.csv')
homegames

,GM_Num,Date,Team,Home,Opp,W/L,R,RA,Inn,W-L,...,Sunday,game_id,Date_Same_Mkt_Team,Team_Same_Mkt_Team,Net_Wins_Same_Mkt_Team,Win_Per_Same_Mkt_Team,Streak_Same_Mkt_Team,Wins_Same_Mkt_Team,Losses_Same_Mkt_Team,date_diff
977,7,2010-04-12,CHC,1,MIL,W,9,5,NaN,3-4,...,0,CHC2010-04-12,2010-04-11,CHW,-2,0.333333,1,2,4,1
1141,8,2010-04-14,CHC,1,MIL,W,7,6,NaN,4-4,...,0,CHC2010-04-14,2010-04-13,CHW,-2,0.375000,-1,3,5,1
1304,9,2010-04-15,CHC,1,MIL,L,6,8,NaN,4-5,...,0,CHC2010-04-15,2010-04-14,CHW,-1,0.444444,1,4,5,1
1467,10,2010-04-16,CHC,1,HOU,W,7,2,NaN,5-5,...,0,CHC2010-04-16,2010-04-15,CHW,-2,0.400000,-1,4,6,1
1630,11,2010-04-17,CHC,1,HOU,L,3,4,NaN,5-6,...,0,CHC2010-04-17,2010-04-16,CHW,-3,0.363636,-2,4,7,1
1793,12,2010-04-18,CHC,1,HOU,L,2,3,10,5-7,...,1,CHC2010-04-18,2010-04-17,CHW,-4,0.333333,-3,4,8,1
3096,20,2010-04-26,CHC,1,WSN,W-wo,4,3,10,10-10,...,0,CHC2010-04-26,2010-04-25,CHW,-3,0.421053,3,8,11,1
3258,21,2010-04-27,CHC,1,WSN,L,1,3,NaN,10-11,...,0,CHC2010-04-27,2010-04-25,CHW,-3,0.421053,3,8,11,2
3421,22,2010-04-28,CHC,1,WSN,L,2,3,NaN,10-12,...,0,CHC2010-04-28,2010-04-27,CHW,-4,0.400000,-1,8,12,1
3584,23,2010-04-29,CHC,1,ARI,L,5,13,NaN,10-13,...,0,CHC2010-04-29,2010-04-28,CHW,-5,0.380952,-2,8,13,1


In [7]:
def mean_std(cat):
    print(cat + ':')
    for i in Teams:
        TeamTemp = homegames[homegames['Team'] == i]
        TeamTemp = TeamTemp[pd.notnull(TeamTemp[cat])]
        TeamTemp[cat] = TeamTemp[cat].astype(str).astype(int)
        TeamMean = TeamTemp[cat].mean()
        TeamStd = TeamTemp[cat].std()
        print(i + "'s Mean: " + str(TeamMean))
        print(i + "'s Standard Dev: " + str(TeamStd))

In [8]:
mean_std('Attendance')
#mean_std('Win_Per')

Attendance:
CHC's Mean: 36891.974968710885
CHC's Standard Dev: 4080.876292558024
CHW's Mean: 22465.715012722645
CHW's Standard Dev: 6536.423707134375
LAA's Mean: 37858.708798017346
LAA's Standard Dev: 4100.180384667041
LAD's Mean: 44832.69937888199
LAD's Standard Dev: 6405.553003267846
NYM's Mean: 29966.022813688214
NYM's Standard Dev: 6223.064231626648
NYY's Mean: 42032.096815286626
NYY's Standard Dev: 4881.165233741245
OAK's Mean: 20075.015132408575
OAK's Standard Dev: 7939.438976393215
SFG's Mean: 40054.90099009901
SFG's Standard Dev: 3294.46544414653


In [9]:
segment_dummies = pd.get_dummies(homegames['Team'])
homegames = pd.concat([homegames, segment_dummies], axis=1)
homegames.head()

,GM_Num,Date,Team,Home,Opp,W/L,R,RA,Inn,W-L,...,Losses_Same_Mkt_Team,date_diff,CHC,CHW,LAA,LAD,NYM,NYY,OAK,SFG
977,7,2010-04-12,CHC,1,MIL,W,9,5,NaN,3-4,...,4,1,1,0,0,0,0,0,0,0
1141,8,2010-04-14,CHC,1,MIL,W,7,6,NaN,4-4,...,5,1,1,0,0,0,0,0,0,0
1304,9,2010-04-15,CHC,1,MIL,L,6,8,NaN,4-5,...,5,1,1,0,0,0,0,0,0,0
1467,10,2010-04-16,CHC,1,HOU,W,7,2,NaN,5-5,...,6,1,1,0,0,0,0,0,0,0
1630,11,2010-04-17,CHC,1,HOU,L,3,4,NaN,5-6,...,7,1,1,0,0,0,0,0,0,0


In [10]:
homegames.to_csv('homegames.csv')